In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os,time
import glob,sys

from astropy.io import fits
from astropy.visualization import ZScaleInterval

from RetrieveSource import *
from EstimateBackground import *
from TractorTools import *
from Heasarc import *
from PSF import *

In [2]:
def vmin(data):
    return ZScaleInterval().get_limits(data)[0]
def vmax(data):
    return ZScaleInterval().get_limits(data)[1]

In [5]:
#path = '../Data/UV/SMC'
observation_id = 40430

def get_hdu_cat_from_file(gal,observation_id):
    # TO DO: 
    # Need to update this function so it does all segments

    filenames = glob.glob((f'../Data/UV/{gal}/*{observation_id}*.img.gz'))
    print(f"%s files found for id {observation_id}" % len(filenames))

    hdus = [fits.open(file)[1] for file in filenames]
    #data = [fits.open(file)[0] for file in filenames]
    names = [file.split('/')[-1].split('.')[0] for file in filenames]
    catfiles = [f"../Data/UV/{gal}/{name}.full.dat" for name in names]

    return hdus,catfiles
        

def model_field(gal,observation_id):
    
    print('Attempting ObsID: ', observation_id)
    t1 = time.time()
    
    # If this obsid has not been run before make it it's own folder
    path = f"../Results/Photometry/{observation_id}"
    if not os.path.exists(path):
        os.mkdir(path)   
        
    # To save time, retrieve sources has been run on most fields saving a catalog of sources per field 
    # Load the right sources within the observation_id field
    hdu,catname = get_hdu_cat_from_file(gal,observation_id) 

    #if len(hdu) < 3: continue
    
    meta = {}; bkgd = {}; TractorObjects = {}; 
    Magnitudes = {}; by_filter = {}


    for i,h in enumerate(hdu):
        uvfilter = h.header['FILTER']
        shape = np.shape(h.data)
    
        # Step 1: Get Zaritsky Coordinates and Catalog 
        tz = time.time()
    
        print('---Getting Catalog---')

        catalog = catname[i]
        print(catalog)
        meta[uvfilter] = get_meta().with_hdu(h,
                                             optical_catalog=catalog,
                                             xdim=[0,shape[1]],
                                             ydim=[0,shape[0]],
                                             fits_origin=0)
    
        print("Time To Run Retrieve Source: ",(time.time()-tz)/60)
    

    
        # Step 2: Error Estimation
        bkgd[uvfilter] = BkgdEstimator(meta[uvfilter],n_pix = [20,20])
    

        # Step 3: Calculate PSF
        pix_scale = np.abs(meta[uvfilter].cdelt)*3600.
        psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                             uvfilter = uvfilter, 
                             width = 23).psf
    
    
        # Step 4: Run Tractor 
        t2 = time.time()
        print('---Starting to Run Tractor---')
        fwhm = 2.5
        # 2.355 is the conversion from fwhm to sigma
        sigma =  fwhm/2.355
        # Restrict by a factor of 4.
        sigma = sigma/4.
        
        exp_str = meta[uvfilter].exposure_time
        
        TractorObjects[uvfilter] = PhotometryTools(meta[uvfilter],
                            psf_filename = psf_object,
                            fits_table_sname = f"{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.fits", # Should change 1_1 to include all segments!
                            background = bkgd[uvfilter],
                            fit_positions = sigma,
                            threshold = 1.5)
        print("Time To Run Tractor: ",(time.time()-t2)/60)
    
        # Step 5: Run Heasarc 
        t3 = time.time()
        Magnitudes[uvfilter] = HeasarcRoutines(f"{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.fits",uvfilter)# Should change 1_1 to include all segments!
        print("Time to Run Heasarc: ",(time.time()-t3)/60)
    
        # Step 6: Format CSV
        t4 = time.time()    
    
        # Get csv created in Heasarc 
        df = pd.read_csv(f'{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.csv')# Should change 1_1 to include all segments!

        labels = [uvfilter + '_'+ key for key in df.keys()]
        d = {}
        for old,new in zip(df.keys(),labels):
            d[old] = new
        df = df.rename(columns=d)

        print(f'Joining Optical catalog with {uvfilter}')
        by_filter[uvfilter] = pd.merge(meta[uvfilter].catalog,df,left_on='KEY',right_on=f'{uvfilter}_KEY',how="inner")
        print("Time to Create CSV by Filter: ",(time.time()-t4)/60)
        
    
        by_filter[uvfilter] = by_filter[uvfilter][by_filter[uvfilter][uvfilter+'_MAG_ERR'] < 0.35]

        return TractorObjects

In [6]:
model_field('SMC',40430)

Attempting ObsID:  40430
3 files found for id 40430
---Getting Catalog---
../Data/UV/SMC/sw00040430003um2_sk_40430_3.full.dat


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55894.213704 from DATE-OBS.
Set MJD-END to 55894.216644 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55894.213704 from DATE-OBS.
Set MJD-END to 55894.216644 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55894.213704 from DATE-OBS.
Set MJD-END to 55894.216644 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55894.213704 from DATE-OBS.
Set MJD-END to 55894.216644 from DATE-END'. [astropy.wcs.wcs]


Time To Run Retrieve Source:  0.10513971249262492


Masked Sigma Clipped Stats Original Data: 0.006371793337166309 0.00568508030846715 0.004013084806501865


Masked Sigma Clipped Stats Back Sub Data: 0.0010510022245961228 0.0005784313810861504 0.003604306448701545
22.908366950672878
23
Scaling is 1.003999981732632 ArcSeconds per Pixel
Image needs to be 23 pixels to be 23 ArcSeconds in size.
23
Sum Before Normalization  616.9218729335043
Sum After Normalization  0.9999999999999998
---Starting to Run Tractor---
tractor.psf: failed to import C version of mp_fourier library.  Falling back to python version.


/home/bethany/Desktop/tractor/tractor/lsqr_optimizer.py:495: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  spcols = np.array(spcols)


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
labels = ["Ra","Dec","Umag","e_Umag",
                     "Bmag","e_Bmag","Vmag","e_Vmag","Imag","e_Imag",
                     "Flag","Jmag","e_Jmag","Hmag","e_Hmag","Ksmag","e_Ksmag"]
        
#df = pd.read_csv("../Data/UV/SMC/sw00040430003um2_sk_40430_3.full.dat",sep="\s+",names=labels)

In [ ]:
#df